<a href="https://colab.research.google.com/github/silverCore97/CNN-exercises/blob/main/CNN%203%20%20hours/Cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import layers,Sequential,optimizers,datasets
import tensorflow as tf

tf.random.set_seed(2345)
conv_layers=[
            #unit 1
           layers.Conv2D(64,3,padding='same',activation=tf.nn.relu),
           layers.Conv2D(64,3,padding='same',activation=tf.nn.relu),
           layers.MaxPool2D(strides=2,padding='same'),

           #unit 2
           layers.Conv2D(128,3,padding='same',activation=tf.nn.relu),
           layers.Conv2D(128,3,padding='same',activation=tf.nn.relu),
           layers.MaxPool2D(strides=2,padding='same'),

           #unit 3
           layers.Conv2D(256,3,padding='same',activation=tf.nn.relu),
           layers.Conv2D(256,3,padding='same',activation=tf.nn.relu),
           layers.MaxPool2D(strides=2,padding='same'),

           #unit 4
           layers.Conv2D(512,3,padding='same',activation=tf.nn.relu),
           layers.Conv2D(512,3,padding='same',activation=tf.nn.relu),
           layers.MaxPool2D(strides=2,padding='same'),

           #unit 5
           layers.Conv2D(512,3,padding='same',activation=tf.nn.relu),
           layers.Conv2D(512,3,padding='same',activation=tf.nn.relu),
           layers.MaxPool2D(strides=2,padding='same')
]

#preprocess x,y x to 0-1
#set x to float and divide by 255 - Normalization
#squeeze out the dimension from y and set to int
def preprocess(x,y):
  x=tf.cast(x,dtype=tf.float32)/255.
  y=tf.cast(y,dtype=tf.int32)
  return x,y

conv_net=Sequential(conv_layers)

fc_layers=[
           layers.Dense(256,activation=tf.nn.relu),
           layers.Dense(128,activation=tf.nn.relu),
           layers.Dense(100,activation=None)
           ]
fc_net=Sequential(fc_layers)

conv_net.build(input_shape=[None,32,32,3])

fc_net.build(input_shape=[None,512])

optimizer=optimizers.Adam(lr=1e-4)
# Get dataset
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
y=tf.squeeze(y,axis=1)
y_test=tf.squeeze(y_test,axis=1)




#enter x,y in training function using the layers
train_db=tf.data.Dataset.from_tensor_slices((x,y))
train_db= train_db.shuffle(1000).map(preprocess).batch(64)

test_db=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db= test_db.map(preprocess).batch(64)

sample=next(iter(train_db))

variables=conv_net.trainable_variables+fc_net.trainable_variables
#run the training function for multiple epochs
for epoch in range(10):
  for step, (x,y) in enumerate(train_db):  
    
    with tf.GradientTape() as tape:

      out=conv_net(x)
      out =tf.reshape(out,[-1,512])
      logits=fc_net(out)
      #use 1-hot-encoding on y
      y_onehot=tf.one_hot(y,depth=100)
      #Calculate losses with crossentropy
      loss=tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True)
      loss=tf.reduce_mean(loss)

      
    #Use adam optimizer to apply gradients to the weights
    grads=tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(grads,variables))

    if step%100==0:
      print(epoch,step,"loss:", float(loss))
  
  #test
  total_number=0
  total_correct=0
  for x,y in test_db:
    out=conv_net(x)
    out = tf.reshape(out,[-1,512])
    logits=fc_net(out)
    #Calculate probabilities for each of the 100 categories
    prob=tf.nn.softmax(logits,axis=1)
    # Choose the category with the maximum value
    pred=tf.argmax(prob,axis=1)
    # Recast the answer into tf.int32
    pred=tf.cast(pred,dtype= tf.int32)

    correct=tf.cast(tf.equal(pred,y),dtype=tf.int32)
    correct=tf.reduce_sum(correct)

    total_correct+=int(correct)
    total_number+=x.shape[0]

  accuracy= total_correct/total_number

  print(epoch, "acc", accuracy)






      




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


0 0 loss: 4.6053466796875
0 100 loss: 4.599911212921143
0 200 loss: 4.433727741241455
0 300 loss: 4.238964557647705
0 400 loss: 4.388631820678711
0 500 loss: 4.0995893478393555
0 600 loss: 3.993873357772827
0 700 loss: 3.942411422729492
0 acc 0.0587
1 0 loss: 4.1155290603637695
1 100 loss: 4.218831539154053
1 200 loss: 3.7031092643737793
1 300 loss: 3.967514991760254
1 400 loss: 3.6305854320526123
1 500 loss: 3.9125266075134277
1 600 loss: 3.6464195251464844
1 700 loss: 3.81416654586792
1 acc 0.1156
2 0 loss: 3.84089994430542
2 100 loss: 3.456129550933838
2 200 loss: 3.6624484062194824
2 300 loss: 3.8472816944122314
2 400 loss: 3.279440402984619
2 500 loss: 3.543823719024658
2 600 loss: 3.844877243041992
2 700 loss: 3.525057792663574
2 acc 0.1726
3 0 loss: 3.4385766983032227
3 100 loss: 3.2946395874023438
3 200 loss: 3.4086861610412598
3 300 loss: 3.3527066707611084
3 400 loss: 3.7126102447509766
3 500 loss: 3.338181972503662
3 600 loss: 3.2416892051696777
3 700 loss: 3.315552234649658